In [1]:
from gurobipy import *
import gurobipy as gp

# type declaration
from typing import Dict, List


import pandas as pd

# %load_ext nb_black
%load_ext lab_black

In [2]:
DONT FORGET TO ADD 
THE FUCKING 0 CONSTRAINT111111111111

combinations_oil_refineries_distributioncenters: Dict[tuple, int] = {
    ("T", "N", "P"): 11 + 11,
    ("T", "C", "P"): 7 + 7,
    ("T", "S", "P"): 2 + 5,
    ("T", "N", "A"): 11 + 7,
    ("T", "C", "A"): 7 + 4,
    ("T", "S", "A"): 2 + 3,
    ("CA", "N", "P"): 7 + 11,
    ("CA", "C", "P"): 4 + 7,
    ("CA", "S", "P"): 8 + 5,
    ("CA", "N", "A"): 7 + 7,
    ("CA", "C", "A"): 4 + 4,
    ("CA", "S", "A"): 8 + 3,
}

# quantity constraint

combinations_oil_refineries_distributioncenters_list = list(
    combinations_oil_refineries_distributioncenters.keys()
)


oil_fields: List[str] = ["T", "CA"]
refineries: List[str] = ["NO", "C", "S"]
distribution_centers: List[str] = ["P", "A"]


# oil_field_limits
oil_field_limits: Dict[str, int] = {"CA": 50000, "T": 10000}

# demand limits
demand_limits: Dict[str, int] = {"P": 20000, "A": 25000}


# set the decision function
"""
ADD THE QUANTITY TO THE PRICE!!!

"""
model = gp.Model()
model.ModelSense = gp.GRB.MINIMIZE

# set the objective function
obj = {}
for combi in combinations_oil_refineries_distributioncenters_list:
    obj[combi] = model.addVar(
        lb=0,
        obj=combinations_oil_refineries_distributioncenters[combi],
        name="{}".format(str(combi)).replace(" ", ""),
    )


# CONSTRAINT FOR SUPPLY: essenetially: for every combintion with T at beginning; this is <= 10000; and <= for CA
for oil_field in oil_field_limits.keys():
    lhs = 0
    for objective_r in obj.keys():
        if objective_r[0] == oil_field:
            lhs += obj[objective_r]
    model.addConstr(lhs <= oil_field_limits[oil_field])


# Constraint for demand
for distro in demand_limits.keys():
    lhs = 0
    for objective_r in obj.keys():
        if objective_r[-1] == distro:
            lhs += obj[objective_r]
    model.addConstr(lhs == demand_limits[distro])


model.optimize()
if not model.status == gp.GRB.OPTIMAL:
    print("something went wrong")
print("optimal value", model.objval)
model.printAttr("X")


model.write("problem.lp")
with open("problem.lp") as f:
    print(f.read())




SyntaxError: invalid syntax (640730118.py, line 1)

ERROR:root:Cannot parse: 1:5: DONT FORGET TO ADD 
Traceback (most recent call last):
  File "/home/angelo/Documents/Uni/Courses/Management Science/Management_Science_Code/lib64/python3.10/site-packages/lab_black.py", line 218, in format_cell
    formatted_code = _format_code(cell)
  File "/home/angelo/Documents/Uni/Courses/Management Science/Management_Science_Code/lib64/python3.10/site-packages/lab_black.py", line 29, in _format_code
    return format_str(src_contents=code, mode=FileMode())
  File "src/black/__init__.py", line 1067, in format_str
  File "src/black/__init__.py", line 1077, in _format_str_once
  File "src/black/parsing.py", line 126, in lib2to3_parse
black.parsing.InvalidInput: Cannot parse: 1:5: DONT FORGET TO ADD 


In [ ]:
DONT FORGET TO ADD 
THE FUCKING 0 CONSTRAINT111111111111

In [ ]:
DONT FORGET TO ADD 
THE FUCKING 0 CONSTRAINT111111111111

# DONT FORGET TO ADD 
THE FUCKING 0 CONSTRAINT111111111111

In [3]:
print("Sensitivity Analysis:")
model.printAttr(["X", "Obj", "SAObjLow", "SAObjUP"])
model.printAttr(["RC", "LB", "SALBLow", "SALBUp", "UB", "SAUBLow", "SAUBUp"])
model.printAttr(["Sense", "Slack", "Pi", "RHS", "SARHSLow", "SARHSUp"])

Sensitivity Analysis:

    Variable            X          Obj     SAObjLow      SAObjUP 
----------------------------------------------------------------
('T','N','P')            0           22            7          inf 
('T','C','P')            0           14            7          inf 
('T','S','P')        10000            7         -inf            8 
('T','N','A')            0           18            4          inf 
('T','C','A')            0           11            4          inf 
('T','S','A')            0            5            4          inf 
('CA','N','P')            0           18           11          inf 
('CA','C','P')        10000           11           10           13 
('CA','S','P')            0           13           11          inf 
('CA','N','A')            0           14            8          inf 
('CA','C','A')        25000            8         -inf            9 
('CA','S','A')            0           11            8          inf 

    Variable           RC          

In [24]:
oil_fields = ["T", "CA"]
refineries = ["N", "C", "S"]
distributors = ["P", "A"]
costs = {
    ("T", "N"): 11,
    ("T", "C"): 7,
    ("T", "S"): 2,
    ("CA", "N"): 7,
    ("CA", "C"): 4,
    ("CA", "S"): 8,
    ("N", "P"): 11,
    ("C", "P"): 7,
    ("S", "P"): 5,
    ("N", "A"): 7,
    ("C", "A"): 4,
    ("S", "A"): 3,
}

supply = {"T": 10000, "CA": 50000}
demand = {"P": 20000, "A": 25000}

model = gp.Model()
x = model.addVars(costs, name="direction")
model.setObjective(
    gp.quicksum(costs[o, r] * x[o, r] for o in oil_fields for r in refineries)
    + gp.quicksum(costs[r, d] * x[r, d] for r in refineries for d in distributors)
)

model.addConstrs(
    gp.quicksum(x[o, r] for r in refineries) <= supply[o] for o in oil_fields
)

model.addConstrs(
    gp.quicksum(x[r, d] for r in refineries) == demand[d] for d in distributors
)

model.addConstrs(
    gp.quicksum(x[o, r] for o in oil_fields)
    == gp.quicksum(x[r, d] for d in distributors)
    for r in refineries
)

model.optimize()
if not model.status == gp.GRB.OPTIMAL:
    print("something went wrong")
print("optimal value", model.objval)
model.printAttr("X")

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 12 columns and 24 nonzeros
Model fingerprint: 0x7f989d17
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 5e+04]
Presolve time: 0.00s
Presolved: 7 rows, 12 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7500000e+05   4.500000e+04   0.000000e+00      0s
       5    3.8000000e+05   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.800000000e+05
optimal value 380000.0

    Variable            X 
-------------------------
direction[T,S]        10000 
direction[CA,C]        35000 
direction[C,P]        10000 
direction[S,P]        10000 
direction[C,A]        25000 


In [23]:
print("Sensitivity Analysis:")
model.printAttr(["X", "Obj", "SAObjLow", "SAObjUP"])
model.printAttr(["RC", "LB", "SALBLow", "SALBUp", "UB", "SAUBLow", "SAUBUp"])
model.printAttr(["Sense", "Slack", "Pi", "RHS", "SARHSLow", "SARHSUp"])

Sensitivity Analysis:

    Variable            X          Obj     SAObjLow      SAObjUP 
----------------------------------------------------------------
direction[T,N]            0           11           -3          inf 
direction[T,C]            0            7            0          inf 
direction[T,S]        10000            2         -inf            6 
direction[CA,N]            0            7            1          inf 
direction[CA,C]        35000            4            0            6 
direction[CA,S]            0            8            6          inf 
direction[N,P]            0           11           10          inf 
direction[C,P]        10000            7            6            8 
direction[S,P]        10000            5            3            6 
direction[N,A]            0            7            1            8 
direction[C,A]        25000            4            3            5 
direction[S,A]            0            3            2          inf 

    Variable           RC 

#### i) What would be the impact on the transportation costs if the supply at Texas increased to 15000. Would the optimal solution change? (5 points)



  Constraint        Sense        Slack           Pi          RHS     SARHSLow      SARHSUp 
------------------------------------------------------------------------------------------
          R0            <            0           -4        10000            0        20000

Looking at the lowest table, this shows that the shadowprice of the first constraint (the texas supply limit) is at Pi = -4; with a range of 10000 to 20000 where the constraint on texas is still binding. This suggests 1) that the constraint of 10000 on texas is binding (or limiting).

Subsequently, an increase to 15000 is still in the range; as such, the shadow price will still apply in this case. Follwoing, we can assume that this binding constraint will be utilized further as it would reduce total costs.

The calculation of by how much it reduces cost is simple: we have an increase in supply by 5000 and a reduction in the objective value by -4. This suggests that the total reduction in price would be 20000€ (the objective value is reduced by 20000€ = 380000 -(-4 * 5000) ) = 360000€

#### ii) What would be the impact on the transportation costs if the supply at California decreased to 40000? Would the optimal solution change? (5 points)

    Variable           RC           LB      SALBLow       SALBUp           UB      SAUBLow       SAUBUp 
-------------------------------------------------------------------------------------------------------
direction[T,N]           14            0            0        10000          inf            0          inf 
direction[T,C]            7            0       -10000        10000          inf            0          inf 
direction[T,S]            0            0         -inf        10000          inf        10000          inf 
direction[CA,N]            6            0            0        25000          inf            0          inf 
direction[CA,C]            0            0         -inf        35000          inf        35000          inf 
direction[CA,S]            2            0       -10000        10000          inf            0          inf 
direction[N,P]            1            0       -25000            0          inf            0          inf 
direction[C,P]            0            0         -inf        10000          inf        10000          inf 
direction[S,P]            0            0         -inf        10000          inf        10000          inf 
direction[N,A]            0            0         -inf            0          inf            0          inf 
direction[C,A]            0            0         -inf        25000          inf        25000          inf 
direction[S,A]            1            0       -10000        10000          inf            0          inf 

  Constraint        Sense        Slack           Pi          RHS     SARHSLow      SARHSUp 
------------------------------------------------------------------------------------------

          R1            <        15000            0        50000        35000          inf 

A decrease of supply in California to 40000 will not change the total transportation cost, nor would it reduce the optimal solution.
The reason for this is that California only supplies already a total of 35000 Barels; this means that the supply is already not fully utilized. 
This is shown in the above table. Where the SALBUp for California is at max 35000. As such, it is a nonbinding constraint.
This can also be seein in the lower table, where there is a slack of 15000 (barels) in this case, which means a reduction of supply to simple 40000 would still leave a slack of 5000, meaning that there is no change to be expected based on the specifications of this model.

As such, neither the transportation cost nor the optimal solution of distributing barels is changed. 


An increase in supply via California will not change the total transport costs simply ebcasue california supply is non-binding contraint; as such, more supply will not lead to more transportation cost in this case 

#### iii) Texago must deliver an additional 10000 barrels to either Pittsburgh or Atlanta. Should they deliver it all to Pittsburgh, all to Atlanta, or split it? Why? Can you calculate the additional cost? (5 points)
  Constraint        Sense        Slack           Pi          RHS     SARHSLow      SARHSUp 
------------------------------------------------------------------------------------------

          R5            =            0            4            0       -35000        15000 
          R6            =            0            6            0       -10000        10000 

BAsed on the constraints of eahc distribution center; we can see that an increase in additional demand by 10000 should be sent to Pitsburg as the shaddow price of sending to ptisburg is only at 4 while the shadow price at Atlanta is 6.

This implies that by sending 10000 barels more to Pitsburg will only increase total transportation cost by 40000 vs 60000 in atlanta.

it may be noted that in order to come to this conclustion we used the ranges provided ro Pitsburg (-35000; 15000) and atlanta (-10000, 10000) based on SARHSLow and SARHSup; this shows than an increase in demand would not be out of bounds as in either case an increase by + 10000 in demand, this owuld still fall into the boundaries given above.
Subsequently, the optimal value would be 420000 (+40000)

#### iv) A new motorway has been built between California and Seatle, reducing the shipping cost to $7 per barrel. What will be the effect on the transportation costs? (5 points)



Here for this answer look at this table:

    Variable            X          Obj     SAObjLow      SAObjUP 
----------------------------------------------------------------
direction[T,N]            0           11           -3          inf 
direction[T,C]            0            7            0          inf 
direction[T,S]        10000            2         -inf            6 
direction[CA,N]            0            7            1          inf 
direction[CA,C]        35000            4            0            6 
direction[CA,S]            0            8            6          inf 
direction[N,P]            0           11           10          inf 
direction[C,P]        10000            7            6            8 
direction[S,P]        10000            5            3            6 
direction[N,A]            0            7            1            8 
direction[C,A]        25000            4            3            5 
direction[S,A]            0            3            2          inf 


The current objective is 8 which is then reduced to 7; as such, we need to look at the SAObjLow = 6 infinity SAObjUP = infinity;
7 lies in between this range. This implies that the objective value will not change. However, while there is a reduction in cost, this does 
not lead to a reduction in transportation cost simply because there are no flows from California to Seattle. Thus the optimal value of 380000 is maintained.



## Q 3

In [3]:
stores = ["A", "B"]

demand = {
    ("A", 0): 45,
    ("A", 1): 20,
    ("A", 2): 20,
    ("A", 3): 25,
    ("A", 4): 15,
    ("A", 5): 28,
    ("A", 6): 15,
    ("B", 0): 8,
    ("B", 1): 12,
    ("B", 2): 23,
    ("B", 3): 30,
    ("B", 4): 12,
    ("B", 5): 10,
    ("B", 6): 33,
}

days = 7
max_demand = 55
purc_price = 200
E_allocation = 20
C_allocation = 5

In [4]:
model = gp.Model()
model.ModelSense = gp.GRB.MINIMIZE

# purchasing day one
Purc_A = model.addVars(days, name="Purc_A", obj=200)
Purc_B = model.addVars(days, name="Purc_B", obj=200)

# "C allocation (5)" from A to B and B to A
cab = model.addVars(days, name="Cab", obj=5)
cba = model.addVars(days, name="Cba", obj=5)

# "E allocation (20)" from A to B and B to A
eab = model.addVars(days, name="Eab", obj=20)
eba = model.addVars(days, name="Eba", obj=20)

# set the function
model.setObjective(
    gp.quicksum(
        ((Purc_A[t] + Purc_B[t]) * 200 + (eab[t] + eba[t]) * 20 + (cab[t] + cba[t]) * 5)
        for t in range(days)
    )
)


Inventory_A = model.addVars(days, name="Inventory_A")
Inventory_B = model.addVars(days, name="Inventory_B")

## inventory management A
# day 1 purchase only; index 0
Inventory_A[0] = Purc_A[0]
# day 2 no purcahse but E allocation: inventory day 2 = Inventory day 1 (t - 1 = 0) + transfers from b at day t - 1 - minus all transfers from a to b and from a to b in the 5 € concerns
Inventory_A[1] = Inventory_A[0] + eba[0] - eab[0] - cab[0]
# day 3 and beyond; is essentially the same as above but now you can also receive
for t in range(2, days):
    Inventory_A[t] = (
        Inventory_A[t - 1] + eba[t - 1] + cba[t - 2] - eab[t - 1] - cab[t - 1]
    )

## the same for B just the other way around!
Inventory_B[0] = Purc_B[0]
Inventory_B[1] = Inventory_B[0] + eab[0] - eba[0] - cba[0]
for t in range(2, days):
    Inventory_B[t] = (
        Inventory_B[t - 1] + eab[t - 1] + cab[t - 2] - eba[t - 1] - cba[t - 1]
    )


# enforce that inventory is at least as big as demand on date t for both inventory a and b
for t in range(days):
    model.addConstr(Inventory_A[t] >= demand[("A", t)], name="Inventory_A")

for t in range(days):
    model.addConstr(Inventory_B[t] >= demand[("B", t)], name="Inventory_B")

# only day one can be used for purchasesing
for t in range(1, days):
    lhs = Purc_A[t] + Purc_B[t]
    model.addConstr(lhs == 0)

# only the last day does not work in this case as there cant be two day transfers (the C transfers)
model.addConstr((cab[days - 1] + cba[days - 1]) == 0)


model.optimize()

print("Total Cost:", model.OBJVAL)
model.printAttr(["x"])

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 21 rows, 56 columns and 184 nonzeros
Model fingerprint: 0x16a8f63d
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 4e+01]
Presolve removed 9 rows and 32 columns
Presolve time: 0.00s
Presolved: 12 rows, 24 columns, 166 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0680000e+04   1.250000e+01   0.000000e+00      0s
       6    1.1265000e+04   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.126500000e+04
Total Cost: 11265.0

    Variable            x 
-------------------------
   Purc_A[0]           45 
   Purc_B[0]           10 
      Cab[0]           11 
      Cab[1]            7 
      Cba[3]            3 
      Eab[0]     

In [5]:
model.write("problem.lp")
with open("problem.lp") as f:
    print(f.read())

\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  200 Purc_A[0] + 200 Purc_A[1] + 200 Purc_A[2] + 200 Purc_A[3]
   + 200 Purc_A[4] + 200 Purc_A[5] + 200 Purc_A[6] + 200 Purc_B[0]
   + 200 Purc_B[1] + 200 Purc_B[2] + 200 Purc_B[3] + 200 Purc_B[4]
   + 200 Purc_B[5] + 200 Purc_B[6] + 5 Cab[0] + 5 Cab[1] + 5 Cab[2]
   + 5 Cab[3] + 5 Cab[4] + 5 Cab[5] + 5 Cab[6] + 5 Cba[0] + 5 Cba[1]
   + 5 Cba[2] + 5 Cba[3] + 5 Cba[4] + 5 Cba[5] + 5 Cba[6] + 20 Eab[0]
   + 20 Eab[1] + 20 Eab[2] + 20 Eab[3] + 20 Eab[4] + 20 Eab[5] + 20 Eab[6]
   + 20 Eba[0] + 20 Eba[1] + 20 Eba[2] + 20 Eba[3] + 20 Eba[4] + 20 Eba[5]
   + 20 Eba[6] + 0 Inventory_A[0] + 0 Inventory_A[1] + 0 Inventory_A[2]
   + 0 Inventory_A[3] + 0 Inventory_A[4] + 0 Inventory_A[5]
   + 0 Inventory_A[6] + 0 Inventory_B[0] + 0 Inventory_B[1]
   + 0 Inventory_B[2] + 0 Inventory_B[3] + 0 Inventory_B[4]
   + 0 Inventory_B[5] + 0 Inventory_B[6]
Subject To
 Inventory_A: Purc_A[0] >= 45
 Inventory_A: Purc_A[0]

In [13]:
model = gp.Model()
model.ModelSense = gp.GRB.MINIMIZE

# purchasing day one
Purc_A = model.addVars(days, name="Purc_A", obj=purc_price)
Purc_B = model.addVars(days, name="Purc_B", obj=purc_price)

# "E allocation (20)" from A to B and B to A
eab = model.addVars(days, name="Eab", obj=E_allocation)
eba = model.addVars(days, name="Eba", obj=E_allocation)

# "C allocation (5)" from A to B and B to A
cab = model.addVars(days, name="Cab", obj=C_allocation)
cba = model.addVars(days, name="Cba", obj=C_allocation)


model.setObjective(
    gp.quicksum(
        (Purc_A[t] + Purc_B[t]) * purc_price
        + (eab[t] + eba[t]) * E_allocation
        + (cab[t] + cba[t]) * C_allocation
        for t in range(0, days)
    )
)


Ia = model.addVars(days, name="Ia")

# inventory management A
for t in range(0, days):
    if t == 0:
        Ia[t] = Purc_A[t]
    elif t == 1:
        Ia[t] = eba[t - 1] + Ia[t - 1] - (eab[t - 1] + cab[t - 1])
    else:
        Ia[t] = eba[t - 1] + cba[t - 2] + Ia[t - 1] - (eab[t - 1] + cab[t - 1])

# inventory management B
Ib = model.addVars(days, name="Ib")

for t in range(0, days):
    if t == 0:
        Ib[t] = Purc_B[t]
    elif t == 1:
        Ib[t] = eab[t - 1] + Ib[t - 1] - (eba[t - 1] + cba[t - 1])
    else:
        Ib[t] = eab[t - 1] + cab[t - 2] + Ib[t - 1] - (eba[t - 1] + cba[t - 1])


model.write("problem.lp")
with open("problem.lp") as f:
    print(f.read())


# model.addConstrs(
#     (Ia[t] >= demand[("A", t)] for t in range(0, days)), name="Inventory A"
# )
# model.addConstrs(
#     (Ib[t] >= demand[("B", t)] for t in range(0, days)), name="Inventory B"
# )

# # only day one can be used for purchasesing
# model.addConstrs((Purc_A[t] + Purc_B[t]) == 0 for t in range(1, days))

# # leave one
# model.addConstr((cab[days - 1] + cba[days - 1]) == 0)


# model.optimize()

# print("Total Cost:", model.OBJVAL)
# model.printAttr(["x"])

\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  200 Purc_A[0] + 200 Purc_A[1] + 200 Purc_A[2] + 200 Purc_A[3]
   + 200 Purc_A[4] + 200 Purc_A[5] + 200 Purc_A[6] + 200 Purc_B[0]
   + 200 Purc_B[1] + 200 Purc_B[2] + 200 Purc_B[3] + 200 Purc_B[4]
   + 200 Purc_B[5] + 200 Purc_B[6] + 20 Eab[0] + 20 Eab[1] + 20 Eab[2]
   + 20 Eab[3] + 20 Eab[4] + 20 Eab[5] + 20 Eab[6] + 20 Eba[0] + 20 Eba[1]
   + 20 Eba[2] + 20 Eba[3] + 20 Eba[4] + 20 Eba[5] + 20 Eba[6] + 5 Cab[0]
   + 5 Cab[1] + 5 Cab[2] + 5 Cab[3] + 5 Cab[4] + 5 Cab[5] + 5 Cab[6]
   + 5 Cba[0] + 5 Cba[1] + 5 Cba[2] + 5 Cba[3] + 5 Cba[4] + 5 Cba[5]
   + 5 Cba[6] + 0 Ia[0] + 0 Ia[1] + 0 Ia[2] + 0 Ia[3] + 0 Ia[4] + 0 Ia[5]
   + 0 Ia[6] + 0 Ib[0] + 0 Ib[1] + 0 Ib[2] + 0 Ib[3] + 0 Ib[4] + 0 Ib[5]
   + 0 Ib[6]
Subject To
Bounds
End



In [ ]:

model.write("problem.lp")
with open("problem.lp") as f:
    print(f.read())

In [ ]:
pd.read_csv("management_science_and_linear_programming/data/processed/sails.csv")